# Causal Rule Ensamble - DEMO


#### Required Packages
- numpy
- pandas
- sklearn (linreg, logreg)
- statsmodels (linreg)

#### Optional Packages
- causallib (IPW)
- econml (CF)
- xbcausalforest (XBCF)



In [1]:
from dataset import generate_syn_dataset

dataset = generate_syn_dataset()
dataset.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,t,y
0,0.245237,0.004161,0.960438,0.510613,0.620284,0.356603,0.786233,0.037064,0.163126,0.542123,1,0.0
1,0.253421,0.877668,0.551902,0.704808,0.967859,0.503308,0.966793,0.582931,0.714093,0.359096,0,1.0
2,0.017862,0.563301,0.257973,0.106179,0.618235,0.086963,0.274478,0.865057,0.769595,0.203459,1,1.0
3,0.083457,0.855360,0.028762,0.428984,0.427204,0.576348,0.784008,0.145274,0.040519,0.836547,0,1.0
4,0.527569,0.988817,0.553115,0.612075,0.006488,0.594614,0.522630,0.952550,0.528445,0.183528,1,1.0


In [12]:
from sklearn.linear_model import LogisticRegression

y = dataset["y"]
t = dataset["t"]
X = dataset.drop(['y', 't'], axis=1)

learner = LogisticRegression()
learner.fit(X, t)
learner.predict_proba(X)[:,1]

array([0.49712309, 0.53130076, 0.46780002, 0.52845135, 0.41525594,
       0.56750801, 0.39733277, 0.24278018, 0.27278024, 0.52134339,
       0.27634355, 0.2673415 , 0.61701628, 0.55627154, 0.24584087,
       0.46948423, 0.25251206, 0.54267036, 0.67530045, 0.57062973,
       0.28835634, 0.5772832 , 0.7132464 , 0.28407825, 0.22079954,
       0.50863733, 0.56408403, 0.52780239, 0.24640286, 0.25869685,
       0.49978521, 0.564448  , 0.29038105, 0.47119324, 0.53224161,
       0.16650131, 0.54462137, 0.44289118, 0.43548606, 0.41215273,
       0.30337415, 0.54522651, 0.20396917, 0.55488236, 0.42769292,
       0.43111497, 0.58726788, 0.56468314, 0.59580695, 0.3450937 ,
       0.56347499, 0.69473746, 0.48042321, 0.58244202, 0.30875913,
       0.75364759, 0.29515483, 0.58574875, 0.48111687, 0.17174097,
       0.15342632, 0.54710198, 0.15961286, 0.4890348 , 0.72215254,
       0.33837594, 0.47297635, 0.87854689, 0.56334213, 0.6470244 ,
       0.59169279, 0.17763428, 0.37916551, 0.41981257, 0.25758

In [35]:
X.loc[:,~(X.describe().loc["mean"]<0.5)].describe()


,x1,x2,x9,x10
count,100.000000,100.000000,100.000000,100.000000
mean,0.525936,0.565668,0.557020,0.541677
std,0.270757,0.272963,0.260726,0.278811
min,0.022324,0.027254,0.020150,0.003031
25%,0.343687,0.322967,0.369767,0.276547
50%,0.541459,0.596208,0.577206,0.550862
75%,0.728546,0.802442,0.759761,0.792266
max,0.981574,0.999611,0.961519,0.999788


In [3]:
from ite import estimate_ite_ipw

y = dataset["y"]
z = dataset["z"]
X = dataset.drop(['y', 'z'], axis=1)

a = x
print(a)


0    -1.485055
1    -2.327055
2     0.000000
3     1.614052
4     1.182206
        ...   
95    2.139964
96   -1.808367
97   -1.859411
98    0.000000
99   -1.368469
Length: 100, dtype: float64


In [9]:
from xbcausalforest import XBCF
import numpy as np
NUM_TREES_PR  = 200
NUM_TREES_TRT = 100

cf = XBCF(
    #model="Normal",
    parallel=True, 
    num_sweeps=50, 
    burnin=15,
    max_depth=250,
    num_trees_pr=NUM_TREES_PR,
    num_trees_trt=NUM_TREES_TRT,
    num_cutpoints=100,
    Nmin=1,
    #mtry_pr=X1.shape[1], # default 0 seems to be 'all'
    #mtry_trt=X.shape[1], 
    tau_pr = 0.6 * np.var(dataset['y'])/NUM_TREES_PR, #0.6 * np.var(y) / /NUM_TREES_PR,
    tau_trt = 0.1 * np.var(dataset['y'])/NUM_TREES_TRT, #0.1 * np.var(y) / /NUM_TREES_TRT,
    alpha_pr= 0.95, # shrinkage (splitting probability)
    beta_pr= 2, # shrinkage (tree depth)
    alpha_trt= 0.95, # shrinkage for treatment part
    beta_trt= 2,
    p_categorical_pr = 0,
    p_categorical_trt = 0,
    standardize_target=True, # standardize y and unstandardize for prediction
         )

In [12]:
y = dataset["y"]
z = dataset["z"]
X = dataset.drop(['y', 'z'], axis=1)

cf.fit(
    x_t=X, # Covariates treatment effect
    x=X, # Covariates outcome (including propensity score)
    y=y,  # Outcome
    z=z, # Treatment group
)

XBCF(num_sweeps = 50, burnin = 15, max_depth = 250, Nmin = 1, num_cutpoints = 100, no_split_penality = 4.605170185988092, mtry_pr = 10, mtry_trt = 10, p_categorical_pr = 0, p_categorical_trt = 0, num_trees_pr = 200, alpha_pr = 0.95, beta_pr = 2.0, tau_pr = 0.0007137000000000001, kap_pr = 16.0, s_pr = 4.0, pr_scale = False, num_trees_trt = 100, alpha_trt = 0.95, beta_trt = 2.0, tau_trt = 0.0002379, kap_trt = 16.0, s_trt = 4.0, trt_scale = False, verbose = False, parallel = True, set_random_seed = False, random_seed = 0, sample_weights_flag = True, a_scaling = True, b_scaling = True)

In [9]:
import numpy as np
a = np.array([1,2,3])
a>int(None)


TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'